# Apprenez une heuristique pour le Go\n,
        "\n",
        "Dans ce TP notÃ©, vous devrez dÃ©ployer des methodes d'apprentissage automatique permettant d'Ã©valuer la qualitÃ© de plateaux de GO.\n",
        "\n",
        "Pour cela, vous disposerez de 41563 exemples de plateau de Go, tous gÃ©nÃ©rÃ©s par gnugo aprÃ¨s quelques coups contre lui mÃªme avec un niveau de difficultÃ© de 0. Par chaque plateau, nous avons lancÃ© 100 matchs de gnugo contre lui mÃªme, toujours avec un niveau 0, et comptÃ© le nombre de victoires de noir et de blanc depuis ce plateau.\n",
        "\n",
        "A noter, chaque \"rollout\" (un rollout et un dÃ©roulement possible du match depuis le plateau de rÃ©fÃ©rence) correspond Ã  des mouvements choisis alÃ©atoirement parmis les 10 meilleurs mouvements possibles, en biasant le choix alÃ©atoire par la qualitÃ© prÃ©dite du mouvement par gnugo (les meilleurs mouvements ont une plus forte probabilitÃ© d'Ãªtre tirÃ©s).\n",
        "\n",
        "Les donnÃ©es dont vous disposez sont brutes. Ce sera Ã  vous de proposer un format adÃ©quat pour utiliser ces donnÃ©es en entrÃ©e de votre rÃ©seau neuronal. \n",
        "\n",
        "## Comment sera Ã©valuÃ© votre modÃ¨le ?\n",

In [4]:
# Import du fichier d'exemples

def get_raw_data_go(file):
    ''' Returns the set of samples from the local file or download it if it does not exists'''
    import gzip, os.path
    import json

    if not os.path.isfile(file):
        print("File", file, "not found, I am downloading it...", end="")
        import urllib.request 
        urllib.request.urlretrieve ("https://www.labri.fr/perso/lsimon/IA-M1/" + file, file)
        print(" Done")

    with gzip.open(file) as fz:
        data = json.loads(fz.read().decode("utf-8"))
    return data

data = get_raw_data_go("samples-9x9.json.gz")
data_to_evaluate = get_raw_data_go("positions-to-evaluate-9x9.json.gz")

print("We have", len(data),"examples with the scores to train, ...")
print("We have", len(data_to_evaluate),"examples to score (but for which we don't know the real values)")

We have 41563 examples with the scores to train, ...
We have 1000 examples to score (but for which we don't know the real values)


In [5]:
def summary_of_example(data, sample_nb):
    ''' Gives you some insights about a sample number'''
    sample = data[sample_nb]
    print("Sample", sample_nb)
    print()
    print("Données brutes en format JSON:", sample)
    print()
    print("The sample was obtained after", sample["depth"], "moves")
    print("The successive moves were", sample["list_of_moves"])
    print("After these moves and all the captures, there was black stones at the following position", sample["black_stones"])
    print("After these moves and all the captures, there was white stones at the following position", sample["white_stones"])
    print("Number of rollouts (gnugo games played against itself from this position):", sample["rollouts"])
    print("Over these", sample["rollouts"], "games, black won", sample["black_wins"], "times with", sample["black_points"], "total points over all this winning games")
    print("Over these", sample["rollouts"], "games, white won", sample["white_wins"], "times with", sample["white_points"], "total points over all this winning games")

print("Exemple sur les données qui ont les résultats visés")
summary_of_example(data,10)

print()
print("Remarquez que, sur les données à prédire, il manque des valeurs :) :")
print(data_to_evaluate[0])

Exemple sur les données qui ont les résultats visés
Sample 10

Données brutes en format JSON: {'depth': 8, 'list_of_moves': ['C6', 'E7', 'C3', 'C8', 'E5', 'G3', 'B7', 'H7'], 'black_stones': ['B7', 'C6', 'E5', 'C3'], 'white_stones': ['C8', 'E7', 'H7', 'G3'], 'rollouts': 100, 'black_wins': 60, 'black_points': 876.0, 'white_wins': 40, 'white_points': 378.0}

The sample was obtained after 8 moves
The successive moves were ['C6', 'E7', 'C3', 'C8', 'E5', 'G3', 'B7', 'H7']
After these moves and all the captures, there was black stones at the following position ['B7', 'C6', 'E5', 'C3']
After these moves and all the captures, there was white stones at the following position ['C8', 'E7', 'H7', 'G3']
Number of rollouts (gnugo games played against itself from this position): 100
Over these 100 games, black won 60 times with 876.0 total points over all this winning games
Over these 100 games, white won 40 times with 378.0 total points over all this winning games

Remarquez que, sur les données à pr